# TODOs
- Add hidden flag to Parts to control whether a Part will be shown initially
- Add color box to parts in navigator
- Fix toggle isssue when mixing toggle and wire
- OrthoViewPoint
- Extend AxisView with id

# Library

In [ ]:
import cadquery as cq
from cq_jupyter import Assembly, Part, DEBUG

import cq_jupyter
cq_jupyter.cq_jupyter.DEBUG = False

In [ ]:
b = cq.Workplane('XY')
box1 = b.box(1,2,3).edges(">X").chamfer(0.1)
box2 = b.transformed(offset=cq.Vector(0, 2, 1)).box(3,2,1).edges(">Z").fillet(0.1)

Assembly(
    Part(box1, "redbox", (1,0,0)), 
    Part(box2, "greenbox", (0,1,0)),
    height=600
)

In [ ]:
from cadquery import Shape, Compound

def _repr_html_(self):
    """
    Jupyter 3D representation support
    """
    return x3d_display(Part(self, "shape0", (1, 1, 0)), export_edges=True)

Shape._repr_html_ = _repr_html_

class Part(object):
    def __init__(self, shape, name, color=(1,1,0)):
        self.shape = shape
        self.name = name
        self.color = color
        
class Assembly(object):
    def __init__(self, *parts, height=400):
        self.parts = parts
        self.height = height
        
    def _repr_html_(self):
        assembly = []
        for part in self.parts:
            # Replace original shape with compound
            assembly.append(Part(Compound.makeCompound(part.shape.objects), part.name, part.color))

        return x3d_display(*assembly, export_edges=True, height=self.height)

# Test

In [ ]:
import cadquery as cq

In [ ]:
b = cq.Workplane('XY')
box1 = b.box(1,2,3).edges(">X").chamfer(0.1)
box2 = b.transformed(offset=cq.Vector(0, 2, 1)).box(3,2,1).edges(">Z").fillet(0.1)

Assembly(
    Part(box1, "redbox", (1,0,0)), 
    Part(box2, "greenbox", (0,1,0)),
    height=600
)


# iPhone Halterung

In [ ]:
length = 70
width = 80
height = 8.5
thickness = 1.5
deviceRadius = 7
innerRadius = (height - 0.5) / 2.0
padding = 11.0 - deviceRadius

b = cq.Workplane('XY')
f = cq.Workplane('XZ')

inner = (
    b.box(length, width, height)
       .edges('|Z').edges('<Y').fillet(deviceRadius)
       .edges("|Y").edges("<Y").fillet(innerRadius)
)

outer = (
    b.box(length + thickness, width + 2 * thickness, height + thickness)
        .edges('|Z').edges('<Y').fillet(deviceRadius + thickness / 2)  
        .edges("|Y").edges("<Y").fillet(innerRadius + thickness/2)
)

front_cutter = (
    f.transformed(offset=cq.Vector(0, 0, -width/2))
    .box(length + 2 * thickness, height + 2 * thickness, 2 * thickness)
)

top_cutter = (
    b.transformed(offset=cq.Vector(0, thickness/2, (height)/2))
    .box(length - deviceRadius, width - deviceRadius + 2* thickness, 2 * thickness)
    .edges('|Z').edges('<Y').fillet(innerRadius)  
)

bottom_cutter = (
    b.transformed(offset=cq.Vector(0, -width/2 + thickness, thickness))
    .box(length - deviceRadius - padding, deviceRadius + 2 * thickness, height + thickness)
    .edges('|Y').edges('<Y').fillet(height/2 + thickness/4)
)

shell = (
    outer
    .cut(inner)
    .cut(front_cutter)
    .cut(top_cutter)
    .cut(bottom_cutter)
)

Assembly(
    Part(shell, "shell", (1,1,0)), 
    Part(front_cutter, "front_cutter", (1,0,0)),
    Part(top_cutter, "top_cutter", (0,1,0)),
    Part(bottom_cutter, "bottom_cutter", (0,0,1)),
    height=800
)

In [ ]:
from cadquery import exporters

In [ ]:
with open("hh.stl", "w") as fd:
    exporters.exportShape(shell, "STL", fd)